In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, Trainer
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import pandas as pd
import os

In [ ]:
# data_path = os.path.join('data', 'scoutie', 'scoutieDataset_russian_jokes_with_vectors.csv') # dataset from world wide web
data_path = os.path.join('data', 'parsed', 'data_jokes.csv') # parsed dataset anekdot.ru

dataset = load_dataset("csv", data_files=data_path)["train"] 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B-Instruct", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
column_name = 'joke'
def tokenize_function(examples):
    tokenized = tokenizer(examples[column_name], truncation=True, max_length=128, padding="max_length")
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


tokenized_dataset = dataset.map(tokenize_function, batched=True)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2.5-1.5B-Instruct",
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="auto"
)



In [ ]:
peft_config = LoraConfig(
    task_type="CAUSAL_LM",  # Для генерации текста
    inference_mode=False,   # Режим обучения
    r=16,                   # Rank матриц LoRA (можно 4-16)
    lora_alpha=16,         # Коэффициент масштабирования
    lora_dropout=0,     # Дропаут для стабильности
    target_modules=[
        "q_proj",          # Проекции запросов
        "k_proj",          # Проекции ключей
        "v_proj",          # Проекции значений
        "o_proj",          # Выходные проекции
        "gate_proj",       # Врата в FFN-слоях
        "up_proj",         # Верхние проекции в FFN
        "down_proj",       # Нижние проекции в FFN
    ],
    bias="none",           # Не трогаем bias-параметры
)

model.add_adapter(adapter_config=peft_config, adapter_name='lora_2')
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

In [ ]:
training_args = TrainingArguments(
    output_dir="./qwen2.5-lora-jokes/v2.0",
    per_device_train_batch_size=8,    # ⚠️ Маленький batch для 4-bit!
    gradient_accumulation_steps=4,    # "Виртуальный" batch_size = 2 * 4 = 8
    num_train_epochs=1,
    learning_rate=2e-4,               # Для LoRA можно повыше (1e-4 — 3e-4)
    optim="paged_adamw_8bit",         # Оптимизатор для 8-bit/4-bit
    logging_steps=10,
    save_steps=500,
    label_names=["labels"],
    fp16=True,                        # Если GPU не поддерживает bfloat16
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
)
trainer.train()